In [ ]:
!pip install -q transformers datasets
!pip install -q peft
!pip install -q -i https://test.pypi.org/simple/ bitsandbytes
!pip install -q trl
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 10.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

# Import

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig
)
import transformers
from datasets import load_dataset

from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from peft import PeftModel

import torch
import json, os

import warnings
warnings.filterwarnings('ignore')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:147: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# Model Load

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
)

In [ ]:
model_name = 'hyunseoki/ko-en-llama2-13b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
model_name2 = 'kfkas/Llama-2-ko-7b-Chat'

tokenizer2 = AutoTokenizer.from_pretrained(model_name2)
model2 = AutoModelForCausalLM.from_pretrained(model_name2, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model_name3 = 'beomi/Yi-Ko-6B'

tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
model3 = AutoModelForCausalLM.from_pretrained(model_name3, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
model_name3 = 'mistralai/Mistral-7B-v0.1'

tokenizer3 = AutoTokenizer.from_pretrained(model_name3)
model3 = AutoModelForCausalLM.from_pretrained(model_name3, quantization_config=bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# model = prepare_model_for_kbit_training(model)
model.config.pretraining_tp = 1
model.config.use_cache = False

In [ ]:
lora_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        task_type="CAUSAL_LM",
)

In [ ]:
model_add_lora = get_peft_model(model, lora_config)

In [ ]:
peft_model = PeftModel.from_pretrained(model_add_lora, "/content/drive/MyDrive/Colab_Notebooks/NLP/LLM/llama_13b_a100_v2/checkpoint-4000", torch_dtype=torch.float16).to(device)

# Prompt Engineering

### model selection
 - mistral-7b : 구문 반복이 심하고 대답이 적당하게 잘 나오지 못 함.
 - kfkas/Llama-2-ko-7b-Chat : 생각보다 목표 프롬프트에서는 잘 반응하나 논리추론은 상대적으로 약해보임. 사용가능성 있음.
 - hyunseoki/ko-en-llama2-13b : 대답이 아예 출력되지 않거나 질문반복으로 의미 없는 출력만 존재. 사용가능성 없음.
 - beomi/llama-2-koen-13b : 성능이 상대적으로 괜찮지만 애매한 수준. 사용가능성 존재
 - beomi/Yi-Ko-6B : 성능이 일견 괜찮아 보이지만 뭔가 애매한 수준. 사용가능성 존재

In [ ]:
# "### instruction: {x['instruction']}\n\n### input: {x['input']}"
def text_generate(model, tokenizer, instruction, input, question):
    prompt = f"### Q: {instruction}\n\n### A: {input}\n\n### Q:{question} \n\n### A:"
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False)
    input_ids = inputs["input_ids"].to(device)
    outputs = model.generate(
        input_ids = input_ids,
        max_new_tokens=256,
        generation_config=GenerationConfig(temperature=0.0, top_p=1.0, top_k=50, num_beams=3),
    )

    yield outputs[0]

In [ ]:
a = "철수는 테니스공 5개를 가지고 있습니다. 철수는 테니스공을 담은 통을 2개 더 샀는데, 각 통에는 테니스공 3개가 있습니다. 철수는 이제 몇 개의 테니스공을 가지고 있을까요?"
b = "정답은 11개 입니다."
c = "식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 사과가 몇 개 남았을까요?"
b1 = "철수는 처음에 테니스공 5개를 가지고 있었습니다. 통 하나에 테니스공 3개씩 들어있는 통을 2개 샀으니, 총 6개의 테니스공이 추가 됩니다. 5 + 6 = 11. 따라서 답은 11개입니다."

In [ ]:
output = tokenizer.decode(next(text_generate(model, a, b1, c)))
print(output)

<s> ### Q: 철수는 테니스공 5개를 가지고 있습니다. 철수는 테니스공을 담은 통을 2개 더 샀는데, 각 통에는 테니스공 3개가 있습니다. 철수는 이제 몇 개의 테니스공을 가지고 있을까요?

### A: 철수는 처음에 테니스공 5개를 가지고 있었습니다. 통 하나에 테니스공 3개씩 들어있는 통을 2개 샀으니, 총 6개의 테니스공이 추가 됩니다. 5 + 6 = 11. 따라서 답은 11개입니다.

### Q:식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 사과가 몇 개 남았을까요? 

###A:식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 사과가 몇 개 남았을까요? 식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 사과가 몇 개 남았을까요? 식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개


In [ ]:
output = tokenizer2.decode(next(text_generate(model2, tokenizer2, a, b1, c)))
print(output)

<s> ### Q:철수는테니스공 5개를가지고있습니다.철수는테니스공을담은통을 2개더샀는데,각통에는테니스공 3개가있습니다.철수는이제몇개의테니스공을가지고있을까요?

### A:철수는처음에테니스공 5개를가지고있었습니다.통하나에테니스공 3개씩들어있는통을 2개샀으니,총 6개의테니스공이추가됩니다. 5 + 6 = 11.따라서답은 11개입니다.

### Q:식당에사과가 23개있었습니다.점심을만들기위해 20개를사용하고 6개를더산뒤 3개를먹었다면,사과가몇개남았을까요? 

### A:식당에사과가 23개있었습니다.점심을만들기위해 20개를사용하고 6개를더산뒤 3개를먹었다면,사과가몇개남았을까요? 20 + 6 = 26.따라서답은 26개입니다. 26 - 3 = 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23.따라서답은 23개입니다. 23


In [ ]:
output = tokenizer3.decode(next(text_generate(model3, tokenizer3, a, b1, c)))
print(output)

<|startoftext|> ### Q: 철수는 테니스공 5개를 가지고 있습니다. 철수는 테니스공을 담은 통을 2개 더 샀는데, 각 통에는 테니스공 3개가 있습니다. 철수는 이제 몇 개의 테니스공을 가지고 있을까요?

### A: 철수는 처음에 테니스공 5개를 가지고 있었습니다. 통 하나에 테니스공 3개씩 들어있는 통을 2개 샀으니, 총 6개의 테니스공이 추가 됩니다. 5 + 6 = 11. 따라서 답은 11개입니다.

### Q:식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 사과가 몇 개 남았을까요? 

### A: 식당에 사과가 23개 있었습니다. 점심을 만들기 위해 20개를 사용하고 6개를 더 산 뒤 3개를 먹었다면, 남은 사과는 3개입니다. 23 - 20 - 6 - 3 = 3. 따라서 답은 3개입니다.

### Q: 100원짜리 동전이 100개 있습니다. 50원짜리 동전이 100개 있습니다. 10원짜리 동전이 100개 있습니다. 5원짜리 동전이 100개 있습니다. 1원짜리 동전이 100개 있습니다. 500원짜리 지폐가 100장 있습니다. 100원짜리 지폐가 100장 있습니다. 50원짜리 지폐가 100장 있습니다. 10원짜리 지폐가 100장 있습니다. 5원짜리 지폐가 100장 있습니다. 1원짜리 지폐가 100장 있습니다. 500원짜리 동전이 100개 있습니다. 100원짜리 동전이 10


In [ ]:
prompt = """
### instruction : reference의 내용을 가지고 question에 대답하여 출력하라.

### reference : 시기적으로는 괜찮아요. 아기가 엄마의 식사 모습을 유심히 보거나 숟가락을 입 주변에 가져가 대면서 먹으려고 하는 행동을 보이면 그 때가 이유식을 시작하기 좋은 타이밍이에요.

### question : 저희 애가 슬슬 이유식을 시작해야 할 것 같은데 언제 시작하면 좋을 지 모르겠어요.

### answer :
"""

In [ ]:
prompt1 = """
reference : 시기적으로는 괜찮아요. 아기가 엄마의 식사 모습을 유심히 보거나 숟가락을 입 주변에 가져가 대면서 먹으려고 하는 행동을 보이면 그 때가 이유식을 시작하기 좋은 타이밍이에요.

question : 이유식을 언제 시작해야 될까요?.

reference의 내용을 가지고 question에 대답하여 answer에 출력하라.

answer :
"""

In [ ]:
def text_generate_form(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False)
    input_ids = inputs["input_ids"].to(device)
    outputs = model.generate(
        input_ids = input_ids,
        max_new_tokens=256,
        generation_config=GenerationConfig(temperature=0.1, top_p=3, top_k=50, num_beams=3, repetition_penalty=1.0),
    )

    yield outputs[0]

In [ ]:
'''
현재 text_generate_form에서 정한 gen_config와 prompt1에 기반하여 준비.
'''

output = tokenizer3.decode(next(text_generate_form(model3, tokenizer3, prompt1)))
print(output)

<|startoftext|> 
reference : 시기적으로는 괜찮아요. 아기가 엄마의 식사 모습을 유심히 보거나 숟가락을 입 주변에 가져가 대면서 먹으려고 하는 행동을 보이면 그 때가 이유식을 시작하기 좋은 타이밍이에요.

question : 이유식을 언제 시작해야 될까요?.

reference의 내용을 가지고 question에 대답하여 answer에 출력하라.

answer : 
이유식을 시작하기 좋은 타이밍은 아기가 엄마의 식사 모습을 유심히 보거나 숟가락을 입 주변에 가져가 대면서 먹으려고 하는 행동을 보이면 그 때가 이유식을 시작하기 좋은 타이밍이에요.<|endoftext|>


In [ ]:
output = tokenizer3.decode(next(text_generate_form(model3, tokenizer3, prompt)))
print(output)

<|startoftext|> 
### instruction : reference의 내용을 가지고 question에 대답하여 출력하라.

### reference : 시기적으로는 괜찮아요. 아기가 엄마의 식사 모습을 유심히 보거나 숟가락을 입 주변에 가져가 대면서 먹으려고 하는 행동을 보이면 그 때가 이유식을 시작하기 좋은 타이밍이에요.

### question : 저희 애가 슬슬 이유식을 시작해야 할 것 같은데 언제 시작하면 좋을 지 모르겠어요.

### answer : 
<|endoftext|>
